# core

> Fill in a module description here

In [ ]:
#| default_exp phenotype

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
# chewc/pheno.py

from typing import Optional
from functools import partial

import jax
import jax.numpy as jnp
from jaxtyping import Array, Float

from chewc.population import Population
from chewc.trait import TraitCollection, _calculate_gvs_vectorized_alternative # IMPORT FIXED


@partial(jax.jit, static_argnames=("ploidy",))
def _set_pheno_internal(
    key: jax.random.PRNGKey,
    pop: Population,
    traits: TraitCollection,
    ploidy: int,
    h2: Float[Array, "nTraits"],
    cor_e: Optional[Float[Array, "nTraits nTraits"]] = None,
) -> Population:
    """Internal, JITted function to set phenotypes."""
    n_traits = traits.n_traits
    if cor_e is None:
        cor_e = jnp.identity(n_traits)

    # 1. Calculate genetic values (now uses the imported function)
    gvs = _calculate_gvs_vectorized_alternative(pop, traits, ploidy)

    # 2. Calculate environmental noise
    var_g = jnp.var(gvs, axis=0)
    var_e = (var_g / h2) - var_g
    cov_e = jnp.diag(jnp.sqrt(jnp.maximum(var_e, 0))) @ cor_e @ jnp.diag(jnp.sqrt(jnp.maximum(var_e, 0)))
    
    environmental_noise = jax.random.multivariate_normal(
        key, jnp.zeros(n_traits), cov_e, (pop.nInd,)
    )

    pheno = gvs + environmental_noise

    # 3. Update population object with all new values
    return pop.replace(
        pheno=pheno,
        bv=gvs - traits.intercept # Assuming additive model for now
    )

def set_pheno(
    key: jax.random.PRNGKey,
    pop: Population,
    traits: TraitCollection,
    ploidy: int,
    h2: Float[Array, "nTraits"],
    cor_e: Optional[Float[Array, "nTraits nTraits"]] = None,
) -> Population:
    """JIT-compatible wrapper to set phenotypes."""
    return _set_pheno_internal(key, pop, traits, ploidy, h2, cor_e)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()